In [2]:
import pandas as pd
import numpy as np

import tqdm
import os

from pathlib import Path
from Levenshtein import distance

In [3]:
import cv2
import os
import tqdm

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
import torch

from torchvision import transforms as tr
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import center_crop, to_pil_image

from torchvision import models
from torch import nn

from einops import rearrange

from torchcam.methods import SmoothGradCAMpp
from torchcam.utils import overlay_mask

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
PICTURES_DIR = "/home/dude/tongue-net/data/pictures"
MASKS_DIR = "/home/dude/tongue-net/data/masks"
CHECK_DIR = "/home/dude/tongue-net/data/check"

In [7]:
def walk(path: str):
    start = None
    result = []
    for root, _, files in os.walk(path):
        if start is None:
            start = root
            
        for file in files:
            if file.lower().endswith(("png", "jpg", "jpeg", "bmp", "heic")):
                result.append((root.lstrip(start), file))

    return result

In [8]:
pictures_df = pd.DataFrame(data=walk(PICTURES_DIR),
                           columns=["Root", "Name"])
masks_df = pd.DataFrame(data=walk(MASKS_DIR),
                           columns=["Root", "Name"])

In [9]:
pictures_df.head()

,Root,Name
0,Доброкачественные новообразования языка,2_фиброма 6 пациент.jpg
1,Доброкачественные новообразования языка,2_Ретенционная киста корня языка.JPG
2,Доброкачественные новообразования языка,2_лимфангиома 2 пациент 2 фото.jpg
3,Доброкачественные новообразования языка,2_пиогенная гранулема.JPG
4,Доброкачественные новообразования языка,2_венозная мальформация.JPG


In [10]:
masks_df.head()

,Root,Name
0,Рак языка/№18,5_2024-02-06_22-46-55_(4).jpg
1,Рак языка/№18,5_2024-02-06 22-46-55 (2).jpg
2,Рак языка/№18,5_2024-02-06 22-46-55.jpg
3,Рак языка/№18,5_2024-02-06 22-46-55 (5).jpg
4,Рак языка/№18,5_2024-02-06_22-46-55_(3).jpg


# Image2Mask correspondence

In [11]:
# Mask dir contains Roots from Images
assert set(masks_df["Root"]) == set(pictures_df.loc[pictures_df["Root"].isin(set(masks_df["Root"]))]["Root"])

In [12]:
comparison_df = masks_df.groupby("Root").count().join(
    pictures_df.groupby("Root").count(),
    lsuffix="_mask",
    rsuffix="_picture"
)

print(comparison_df.loc[(comparison_df["Name_mask"] != comparison_df["Name_picture"])])

                                                    Name_mask  Name_picture
Root                                                                       
Здоровый язык/№1                                           18             9
Здоровый язык/№10                                          10             5
Здоровый язык/№11                                           8             4
Здоровый язык/№12                                           8             4
Здоровый язык/№18                                          10             5
Здоровый язык/№19                                           7             6
Здоровый язык/№20                                           5             4
Здоровый язык/№21                                           5             4
Здоровый язык/№22                                           5             4
Здоровый язык/№30                                          16             8
Здоровый язык/№31                                           8             4
Здоровый язы

In [13]:
pic_names = []
for _, mask_row in masks_df.iterrows():
    pics = pictures_df.loc[pictures_df["Root"] == mask_row["Root"]]

    distances = []
    for _, pic_row in pics.iterrows():
        distances.append(distance(
            mask_row["Name"].replace("mask", "").replace(" ", "_"),
            pic_row["Name"].replace(" ", "_")))

    argmin = np.argmin(distances)
    pic_names.append(pics.iloc[argmin]["Name"])

masks_df["Corresponding"] = pic_names

In [14]:
masks_df.head()

,Root,Name,Corresponding
0,Рак языка/№18,5_2024-02-06_22-46-55_(4).jpg,5_2024-02-06 22-46-55 (4).jpeg
1,Рак языка/№18,5_2024-02-06 22-46-55 (2).jpg,5_2024-02-06 22-46-55 (2).jpeg
2,Рак языка/№18,5_2024-02-06 22-46-55.jpg,5_2024-02-06 22-46-55.jpeg
3,Рак языка/№18,5_2024-02-06 22-46-55 (5).jpg,5_2024-02-06 22-46-55 (5).jpeg
4,Рак языка/№18,5_2024-02-06_22-46-55_(3).jpg,5_2024-02-06 22-46-55 (3).jpeg


In [ ]:
transfromer = tr.Compose([
    tr.ToPILImage(),
    tr.ToTensor(),
])

tqdrator = tqdm.tqdm(masks_df.iterrows(), total=len(masks_df))
for _, row in tqdrator:
    image = cv2.imread(f"{PICTURES_DIR}/{row['Root']}/{row['Corresponding']}")[:, :, [2, 1, 0]]
    mask = cv2.imread(f"{MASKS_DIR}/{row['Root']}/{row['Name']}")[:, :, [2, 1, 0]]

    image = transfromer(image)
    mask = transfromer(mask)

    if mask.shape[1:] != image.shape[1:]:
        print(f"{PICTURES_DIR}/{row['Root']}/{row['Corresponding']}", "-- different shapes")
        mask = tr.Resize(image.shape[1:])(mask)

    image = image * mask
    image = rearrange(image, "layer height width -> height width layer")

    if not os.path.exists(f"{CHECK_DIR}/{row['Root']}"):
        os.makedirs(f"{CHECK_DIR}/{row['Root']}")
    
    cv2.imwrite(f"{CHECK_DIR}/{row['Root']}/{row['Name']}",
                (image * 255).numpy().astype(int))

    tqdrator.set_description(f"{row['Root']}/{row['Name']}")

In [15]:
problems = [
    "Здоровый язык/№19/0_2023-11-22 10-53-50 (6)_mask.jpg", # extra
    "Здоровый язык/№20/0_2023-11-22 10-55-56 (4)_mask.jpg", # extra
    "Здоровый язык/№21/0_2023-11-22 11-13-29 (4)_mask.jpg", # extra
    "Здоровый язык/№22/0_2023-11-22 11-15-39 (4)_mask.jpg", # extra
    "Здоровый язык/№33/0_2023-11-22_13-27-59_2_mask.jpg", # extra
    "Здоровый язык/№34/0_2023-11-22_13-29-55_mask.jpg", # not found
    "Здоровый язык/№39/0_2023-11-24_15-12-30_(4)_mask.jpg", # with teeth
    "Рак языка/№18/5_2024-02-06 22-46-55 (3).jpg", # no cancer
    "Рак языка/№18/5_2024-02-06 22-46-55 (4).jpg", # no cancer
    "Рак языка/№18/5_2024-02-06 22-46-55 (5).jpg", # no cancer
    "Рак языка/№18/5_2024-02-06_22-46-55_(3).jpg", # no cancer
    "Рак языка/№18/5_2024-02-06_22-46-55_(4).jpg", # no cancer
    "Рак языка/№18/5_2024-02-06_22-46-55_(5).jpg", # no cancer
    "Рак языка/ГКОБ №1. ОГШ. Без сегментации/5_f2043ed9-055b-457d-94fb-217b87237e5f.jpg", # extra
    "Рак языка/ГКОБ №1. ОГШ. Без сегментации/5_Рак языка 2-1_mask.png", # strange
    "Рак языка/ГКОБ №1. ОГШ. Без сегментации/5_Рак языка, с переходом на дно полости рта._mask.jpg", # strange
    "Рак языка/ГКОБ №1. ОГШ. Без сегментации/5_Рак языка, язвенная форма_mask.png", # tight segmentation
    "Рак языка/ГКОБ №1. ОГШ. Без сегментации/5_Рак языка. Экзофитная форма_mask.jpg", # no segmentation
    "Рак языка/ГКОБ №1. ОГШ. Без сегментации/5_рак_mask.png", # no segmentation
    "Рак языка/ГКОБ №1. ОГШ. Без сегментации/5_рак1_mask.png", # no segmentation
]

In [16]:
masks_df = masks_df.loc[~(masks_df["Root"] + "/" + masks_df["Name"]).isin(problems)]

In [17]:
masks_df.to_csv(f"{MASKS_DIR}/correspondence.tsv", sep="\t", index=None)

# Annotation

In [18]:
pictures_df["Class"] = None
pictures_df.loc[pictures_df["Root"].str.startswith("Здоровый язык"), "Class"] = 0

pictures_df.loc[pictures_df["Root"].str.startswith("Рак языка") & ~pictures_df["Name"].str.startswith("5"), "Class"] = 0
pictures_df.loc[pictures_df["Root"].str.startswith("Рак языка") & pictures_df["Name"].str.startswith("5"), "Class"] = 1

# pictures_df.to_csv(f"{PICTURES_DIR}/annotation.tsv", sep="\t", index=None)

In [19]:
pictures_df.Class.value_counts()

Class
0    343
1    238
Name: count, dtype: int64

In [20]:
centers_df = pd.read_csv(f"{PICTURES_DIR}/tumor_coords.tsv", sep="\t")

centers_df = centers_df.rename(columns={
    "ПУТЬ": "Path",
    "КООРДИНАТЫ": "Coords"
})

centers_df["Path"] = centers_df.Path.str.split("/").str[1:].str.join("/")

centers_df = centers_df.set_index("Path")

In [21]:
pictures_df["Path"] = pictures_df["Root"] + "/" + pictures_df["Name"]
pictures_df = pictures_df.set_index("Path")

In [22]:
tumor_index = pictures_df.loc[pictures_df.Class == 1].index

print(len(centers_df.index), len(tumor_index))
print(len(set(centers_df.index) - set(tumor_index)),
      len(set(centers_df.index) & set(tumor_index)),
      len(set(tumor_index) - set(centers_df.index)))

136 238
23 113 125


In [23]:
pictures_df = pictures_df.join(centers_df, how="left")

In [24]:
pictures_df.to_csv(f"{PICTURES_DIR}/annotation.tsv", sep="\t", index=None)

# Check

In [25]:
pictures_df

,Root,Name,Class,Coords
Path,,,,
Доброкачественные новообразования языка/2_фиброма 6 пациент.jpg,Доброкачественные новообразования языка,2_фиброма 6 пациент.jpg,None,NaN
Доброкачественные новообразования языка/2_Ретенционная киста корня языка.JPG,Доброкачественные новообразования языка,2_Ретенционная киста корня языка.JPG,None,NaN
Доброкачественные новообразования языка/2_лимфангиома 2 пациент 2 фото.jpg,Доброкачественные новообразования языка,2_лимфангиома 2 пациент 2 фото.jpg,None,NaN
Доброкачественные новообразования языка/2_пиогенная гранулема.JPG,Доброкачественные новообразования языка,2_пиогенная гранулема.JPG,None,NaN
Доброкачественные новообразования языка/2_венозная мальформация.JPG,Доброкачественные новообразования языка,2_венозная мальформация.JPG,None,NaN
...,...,...,...,...
Предраковые заболевания /Лейкоплакия языка /3_2023-11-08 11-19-09 (3).jpeg,Предраковые заболевания /Лейкоплакия языка,3_2023-11-08 11-19-09 (3).jpeg,None,NaN
Предраковые заболевания /Лейкоплакия языка /0_2023-11-08 11-19-09.jpeg,Предраковые заболевания /Лейкоплакия языка,0_2023-11-08 11-19-09.jpeg,None,NaN
Предраковые заболевания /Лейкоплакия языка /0_2023-11-08 11-19-09 (1).jpeg,Предраковые заболевания /Лейкоплакия языка,0_2023-11-08 11-19-09 (1).jpeg,None,NaN


In [26]:
masks_df["Path"] = masks_df["Root"] + "/" + masks_df["Corresponding"]
masks_df = masks_df.set_index("Path")

In [27]:
check_df = pictures_df.join(masks_df.drop(columns=["Root", "Corresponding"]).rename(columns={"Name": "Mask"}), how="left")

In [ ]:
transfromer = tr.Compose([
    tr.ToPILImage(),
    tr.ToTensor(),
])

query_df = check_df.loc[~check_df["Mask"].isna()]
tqdrator = tqdm.tqdm(query_df.iterrows(), total=len(query_df))

for _, row in tqdrator:
    image = cv2.imread(f"{PICTURES_DIR}/{row['Root']}/{row['Name']}")[:, :, [2, 1, 0]]
    mask = cv2.imread(f"{MASKS_DIR}/{row['Root']}/{row['Mask']}")[:, :, [2, 1, 0]]

    image = transfromer(image)
    mask = transfromer(mask)

    if mask.shape[1:] != image.shape[1:]:
        print(f"{PICTURES_DIR}/{row['Root']}/{row['Mask']}", "-- different shapes")
        mask = tr.Resize(image.shape[1:])(mask)

    image = image * mask
    image = rearrange(image, "layer height width -> height width layer")

    fig, ax = plt.subplots(1, 1, figsize=(15, 15))
    ax.imshow(image)

    tqdrator.set_description(f"{row['Root']}/{row['Name']} {row['Coords']}") 
    
    if row["Coords"] is not None and row["Coords"] == row["Coords"]:
        x, y = map(lambda x: int(x.lstrip().rstrip()), row["Coords"].split(","))
        ax.add_patch(plt.Circle((x, y), 20, color="yellow", alpha=0.7, fill=True))

    if not os.path.exists(f"{CHECK_DIR}/{row['Root']}"):
        os.makedirs(f"{CHECK_DIR}/{row['Root']}")
    
    fig.savefig(f"{CHECK_DIR}/{row['Root']}/{row['Name']}")

Рак языка/№8/5_2023-12-13 09-39-21 (5).jpeg 1959, 897:  15%|▏| 86/560 [02:42<16: